In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import dspy

In [4]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("test")
mlflow.dspy.autolog()


2025/12/12 10:26:17 WARNING mlflow.utils.autologging_utils: MLflow dspy autologging is known to be compatible with 2.5.41 <= dspy, but the installed version is 3.1.0b1. If you encounter errors during autologging, try upgrading / downgrading dspy to a compatible version, or try upgrading MLflow.


In [5]:
import random
from dspy.datasets import DataLoader

kwargs = dict(fields=("claim", "supporting_facts", "hpqa_id", "num_hops"), input_keys=("claim",))
hover = DataLoader().from_huggingface(dataset_name="vincentkoc/hover-parquet", split="train", trust_remote_code=True, **kwargs)

hpqa_ids = set()
hover = [
    dspy.Example(claim=x.claim, titles=list(set([y["key"] for y in x.supporting_facts]))).with_inputs("claim")
    for x in hover
    if x["num_hops"] == 3 and x["hpqa_id"] not in hpqa_ids and not hpqa_ids.add(x["hpqa_id"])
]

random.Random(0).shuffle(hover)
trainset, devset, testset = hover[:100], hover[100:200], hover[650:]

example = trainset[0]

print("Claim:", example.claim)
print("Pages that must be retrieved:", example.titles)

import ujson
import bm25s
import Stemmer
import os
from dspy.utils import download

# Download and extract data if not exists
if not os.path.exists("wiki.abstracts.2017.jsonl"):
    download("https://huggingface.co/dspy/cache/resolve/main/wiki.abstracts.2017.tar.gz")
    os.system("tar -xzvf wiki.abstracts.2017.tar.gz")

corpus = []
if os.path.exists("wiki.abstracts.2017.jsonl"):
    with open("wiki.abstracts.2017.jsonl") as f:
        for line in f:
            line = ujson.loads(line)
            corpus.append(f"{line['title']} | {' '.join(line['text'])}")

stemmer = Stemmer.Stemmer("english")
corpus_tokens = bm25s.tokenize(corpus, stopwords="en", stemmer=stemmer)

retriever = bm25s.BM25(k1=0.9, b=0.4)
retriever.index(corpus_tokens)

DOCS = {}

def search(query: str, k: int) -> list[str]:
    tokens = bm25s.tokenize(query, stopwords="en", stemmer=stemmer, show_progress=False)
    results, scores = retriever.retrieve(tokens, k=k, n_threads=1, show_progress=False)
    
    retrieved_docs = [corpus[doc] for doc in results[0]]
    
    for doc_str in retrieved_docs:
         if " | " in doc_str:
             title, text = doc_str.split(" | ", 1)
             DOCS[title] = text
             
    return retrieved_docs

# search("France", 5)

def search_wikipedia(query: str) -> list[str]:
    """Returns top-5 results and then the titles of the top-5 to top-30 results."""

    topK = search(query, 30)
    titles, topK = [f"`{x.split(' | ')[0]}`" for x in topK[5:30]], topK[:5]
    return topK + [f"Other retrieved pages have titles: {', '.join(titles)}."]

def lookup_wikipedia(title: str) -> str:
    """Returns the text of the Wikipedia page, if it exists."""

    if title in DOCS:
        return DOCS[title]

    results = [x for x in search(title, 10) if x.startswith(title + " | ")]
    if not results:
        return f"No Wikipedia page found for title: {title}"
    return results[0]

def top5_recall(example, pred, trace=None):
    gold_titles = example.titles
    recall = sum(x in pred.titles[:5] for x in gold_titles) / len(gold_titles)

    # If we're "bootstrapping" for optimization, return True if and only if the recall is perfect.
    if trace is not None:
        return recall >= 1.0
    
    # If we're just doing inference, just measure the recall.
    return recall

evaluate = dspy.Evaluate(devset=devset, metric=top5_recall, num_threads=16, display_progress=True, display_table=5)

instructions = "Find all Wikipedia titles relevant to verifying (or refuting) the claim."
signature = dspy.Signature("claim -> titles: list[str]", instructions)
react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=20)

/Users/isaac/projects/worktrees/interp/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Claim: This director is known for his work on Miss Potter. The Academy of Motion Picture Arts and Sciences presents the award in which he was nominated for his work in "Babe".
Pages that must be retrieved: ['Academy Award for Best Director', 'Miss Potter', 'Chris Noonan']


In [6]:
import warnings
warnings.filterwarnings("ignore", message=r"(?s)Pydantic serializer warnings:.*StreamingChoices")

In [7]:
from dspy.datasets import DataLoader
from datasets import load_dataset


student_lm = dspy.LM("openrouter/qwen/qwen3-8b")
teacher_lm = dspy.LM("openai/gpt-5")
dspy.configure(lm=student_lm)
# student_lm("this is a test")

In [9]:
react(claim="David Gregory was born in 1625.").titles[:3]

/Users/isaac/projects/worktrees/interp/.venv/lib/python3.13/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...rt with the search.\n'}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...finish_reason': 'stop'}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/Users/isaac/projects/worktrees/interp/.venv/lib/python3.13/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...nswer is to finish.\n'}

['David Gregory (mathematician)', 'David Gregory (journalist)']

Trace(trace_id=tr-31734ff3467d6929a96590472ed137d9)

In [15]:
def safe_react(claim: str):
    try:
        return react(claim=claim)
    except Exception as e:
        return dspy.Prediction(titles=[])

eval_result = evaluate(safe_react)
print(eval_result)


Average Metric: 57.67 / 100 (57.7%): : 101it [05:35,  3.32s/it]                       

2025/12/11 09:31:35 INFO dspy.evaluate.evaluate: Average Metric: 57.666666666666664 / 100 (57.7%)


,claim,example_titles,trajectory,reasoning,pred_titles,top5_recall
0,The Church of England's movement that inspired the Trinity Episcop...,"[Oxford Movement, Trinity Episcopal Church (Houghton, Michigan), S...","{'thought_0': ""To verify the claim, I need to identify the specifi...",The claim states that the Church of England's movement inspiring T...,"[Oxford Movement, Samuel Rickards]",✔️ [0.667]
1,"Red, White & Crüe and this athlete both fight. The french fighter ...","[Bobby Stewart, Red, White &amp; Crüe, Mike Tyson]","{'thought_0': 'I need to find Wikipedia pages that mention ""Red, W...","The claim mentions ""Red, White & Crüe"" and a French fighter traine...","[Red, White & Crüe, Bobby Stewart, Mötley Crüe, Mike Tyson, French...",✔️ [0.667]
2,The writer/director/actor from Glen or Glenda and Fernand Rivers s...,"[Glen or Glenda, Ed Wood, Fernand Rivers]",{'thought_0': 'I need to verify the claim by identifying the indiv...,"The claim suggests that the creator of ""Glen or Glenda"" and ""Ferna...","[Ed Wood, Fernand Rivers]",✔️ [0.667]
3,The film by Sandi Sissel was released before The End of Suburbia.,"[The End of Suburbia, Sandi Sissel, Chicken Ranch (film)]",{'thought_0': 'I need to find the release date of Sandi Sissel\'s ...,"The claim states that a film by Sandi Sissel was released before ""...","[Sandi Sissel, Chicken Ranch, Paul Jacobs and the Nuclear Gang, Th...",✔️ [0.667]
4,The actor who played captain hook in the live production with Tayl...,"[Taylor Louderman, Christopher Walken, Peter Pan Live!]",{'thought_0': 'I need to identify the actor who played Captain Hoo...,The claim is verified. Taylor Louderman was in the 2013 NBC live p...,"[Taylor Louderman, Peter Pan Live!, Christopher Walken, The Deer H...",✔️ [1.000]


EvaluationResult(score=57.67, results=<list of 100 results>)


In [30]:
mlflow.dspy.autolog()

2025/12/11 10:16:00 WARNING mlflow.utils.autologging_utils: MLflow dspy autologging is known to be compatible with 2.5.41 <= dspy, but the installed version is 3.1.0b1. If you encounter errors during autologging, try upgrading / downgrading dspy to a compatible version, or try upgrading MLflow.


In [ ]:
import dspy

from dspy.teleprompt.teleprompt import Teleprompter
from typing import Any

class NoLabelOptimizer(Teleprompter):
    def get_last_traces(self, eval_result):
        results = [result[1][1][-1] for result in eval_result.results]
        result_dict = [{"trajectory": result[1], "prediction": result[2]} for result in results]
        return result_dict
    
    def compile(self, student: dspy.Module, *, trainset: list[dspy.Example], teacher_lm: dspy.LM, bsz=20) -> dspy.Module:
        def wrap_student(student_prog):
            def wrapped_student(*args, **kwargs):
                with dspy.context(trace=[]):
                    result = student_prog(*args, **kwargs)
                    trace = dspy.settings.trace
                    return [result, trace]
            return wrapped_student
        evaluate = dspy.Evaluate(devset=trainset, metric=lambda e, p, t=None: 0, num_threads=16, display_progress=True)
        eval_result = evaluate(wrap_student(student))
        print(eval_result)

        return student, eval_result

    def get_params(self) -> dict[str, Any]:
        """
        Get the parameters of the teleprompter.

        Returns:
            The parameters of the teleprompter.
        """
        return self.__dict__

optimizer = NoLabelOptimizer()

optimized_react, eval_result = optimizer.compile(react, trainset=trainset[:20], teacher_lm=teacher_lm)


Average Metric: 0.00 / 20 (0.0%): 100%|██████████| 20/20 [02:04<00:00,  6.23s/it]

2025/12/11 09:33:52 INFO dspy.evaluate.evaluate: Average Metric: 0 / 20 (0.0%)



EvaluationResult(score=0.0, results=<list of 20 results>)


In [ ]:
from collections import Counter
import rich
predictions = [result[1] for result in eval_result.results]
# print(type(predictions))
# print(type(predictions[0]))
# print(len(predictions[0]))
rich.print(predictions[0])
c = Counter(type(prediction) for prediction in predictions)
len_c = Counter(len(prediction) for prediction in predictions)
traces = [prediction[1] for prediction in predictions]
final_trajectories = [trace[-1] for trace in traces]
# print(final_trajectories[0])
# print(type(traces[0]))


# rich.print(get_last_traces(eval_result)[0])


[
    Prediction(
    trajectory={'thought_0': 'I need to identify the director of "Miss Potter" and check if they were nominated for
an Academy Award for their work on "Babe".', 'tool_name_0': 'search_wikipedia', 'tool_args_0': {'query': 'Miss 
Potter director'}, 'observation_0': ['Miss Potter | Miss Potter is a 2006 Anglo-American biographical fiction 
family drama film directed by Chris Noonan.  It is a biographical film of children\'s author and illustrator 
Beatrix Potter, and combines stories from her own life with animated sequences featuring characters from her 
stories, such as Peter Rabbit.  Scripted by Richard Maltby, Jr., the director of the Tony-winning Broadway revue, 
"Fosse", the film stars Renée Zellweger in the title role; Ewan McGregor as her publisher and fiancé, Norman Warne;
and Lloyd Owen as solicitor William Heelis.  Emily Watson stars as Warne\'s sister, Millie.  Lucy Boynton also 
stars as the young Beatrix Potter.  It was filmed in St. Peter\'s Square Hammersmith, Cecil Court, Osterley Park, 
Covent Garden, the Isle of Man, Scotland and the Lake District.  "Miss Potter" was released on 29 December 2006 so 
that the film could compete for the 2007 Academy Awards.  The film was intended to be released generally on 12 
January 2007, but Variety.com reported that the Weinstein Company had decided to push a wider release date until 
after the Academy Awards on 25 February 2007.  The date seemed to fluctuate a number of times, but the Weinstein 
Company website ultimately listed its release date as 9 March.  The film received generally positive reviews and 
earned Zellweger her sixth Golden Globe nomination.', 'The Story of Miss Moppet | The Story of Miss Moppet is a 
tale about teasing, featuring a kitten and a mouse, that was written and illustrated by Beatrix Potter.  It was 
published by Frederick Warne & Co for the 1906 Christmas season.  Potter was born in London in 1866, and between 
1902 and 1905 published a series of small format children\'s books with Warne.  In 1906, she experimented with an 
atypical panorama design for "Miss Moppet", which booksellers disliked; the story was reprinted in 1916 in small 
book format.', 'H. C. Potter | Henry Codman Potter (sometimes II or Jr; November 13, 1904 – August 31, 1977) was an
American theatrical producer and director and film director.', 'Sally Potter | Charlotte Sally Potter, OBE (born 19
September 1949) is an English film director and screenwriter.', 'Bärbel Wartenberg-Potter | Bärbel 
Wartenberg-Potter (also known as Bärbel von Wartenberg-Potter) (born 1943) is a German theologian.  After serving 
as director over programs for women and children with the World Council of Churches, teaching theology in Jamaica, 
and serving a pastorate in Stuttgart, Wartenberg-Potter became the president of the .  In 2001 she began a 
seven-year term as the third woman serving as a Bishop of a Lutheran Church in Germany.', "Other retrieved pages 
have titles: `Regis Brodie`, `Centropyge potteri`, `William C. Potter`, `Harry Potter and the Prisoner of Azkaban 
(film)`, `Barron v Potter`, `Bonnie Wright`, `Whorrey Potter and the Sorcerer's Balls`, `Brandon Potter`, `Miss 
Panamá 2016`, `Miss Kentucky USA`, `Miss Peru 2017`, `Peter MacDonald (director)`, `Miss Arizona`, `Melissa 
Potter`, `Tracy Potter`, `Miss Kentucky United States`, `Miss Kamala`, `Miss Canada International`, `Stephen 
Woolfenden`, `Hiam Amani Hafizuddin`, `Miss Teen America`, `Miss Universe Sweden`, `Julia Galloway`, `Joseph E. 
Potter`, `Julie Nauche`."], 'thought_1': 'I need to confirm if Chris Noonan, the director of Miss Potter, was 
nominated for an Academy Award for his work on "Babe".', 'tool_name_1': 'lookup_wikipedia', 'tool_args_1': 
{'title': 'Chris Noonan'}, 'observation_1': 'Chris Noonan (born 14 November 1952) is a Sydney-based Australian 
filmmaker and actor best known for the family film "Babe" (1995), for which he was nominated for both the Academy 
Award for Best Director and Academy

In [8]:
def get_last_traces(eval_result):
    results = [result[1][1][-1] for result in eval_result.results]
    result_dict = [{"trajectory": result[1], "prediction": result[2]} for result in results]
    return result_dict

In [ ]:
import rich

rich.print({"current_react_prompt": react.react.signature.instructions, "current_extraction_prompt": react.extract.predict.signature.instructions})

{
    'current_react_prompt': "Find all Wikipedia titles relevant to verifying (or refuting) the claim.\n\nYou are an
Agent. In each episode, you will be given the fields `claim` as input. And you can see your past trajectory so 
far.\nYour goal is to use one or more of the supplied tools to collect any necessary information for producing 
`titles`.\n\nTo do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and 
also when finishing the task.\nAfter each tool call, you receive a resulting observation, which gets appended to 
your trajectory.\n\nWhen writing next_thought, you may reason about the current situation and plan for future 
steps.\nWhen selecting the next_tool_name and its next_tool_args, the tool must be one of:\n\n(1) search_wikipedia,
whose description is <desc>Returns top-5 results and then the titles of the top-5 to top-30 results.</desc>. It 
takes arguments {'query': {'type': 'string'}}.\n(2) lookup_wikipedia, whose description is <desc>Returns the text 
of the Wikipedia page, if it exists.</desc>. It takes arguments {'title': {'type': 'string'}}.\n(3) finish, whose 
description is <desc>Marks the task as complete. That is, signals that all information for producing the outputs, 
i.e. `titles`, are now available to be extracted.</desc>. It takes arguments {}.\nWhen providing `next_tool_args`, 
the value inside the field must be in JSON format",
    'current_extraction_prompt': 'Find all Wikipedia titles relevant to verifying (or refuting) the claim.'
}

In [ ]:
# mlflow.dspy.autolog(disable=True)

In [ ]:
# ==============
# Initial single step 5x experiment
# ==============
import rich

class ReasonAboutTraces(dspy.Signature):
    """Given a set of trajectories, and the prompt that was used to generate them,
    reason about the traces and how you might improve the prompt from a strategy perspective without overfitting"""
    history: list[str] = dspy.InputField()
    current_state: dict[str, Any] = dspy.InputField()
    deviations_from_specification: str = dspy.OutputField(desc="Deviations from the specification given in the signature of the prompt")
    noticed_patterns: str = dspy.OutputField()
    generic_strategy: str = dspy.OutputField()
    improved_react_prompt: str = dspy.OutputField()
    improved_extraction_prompt: str = dspy.OutputField()

# get 20 trajectories
def wrap_student(student_prog):
    def wrapped_student(*args, **kwargs):
        with dspy.context(trace=[]):
            result = student_prog(*args, **kwargs)
            trace = dspy.settings.trace
            return [result, trace]
    return wrapped_student

instructions = "Find all Wikipedia titles relevant to verifying (or refuting) the claim."
signature = dspy.Signature("claim -> titles: list[str]", instructions)
react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=10)

current_trainset = trainset[:20]
evaluate_trainset = dspy.Evaluate(devset=current_trainset, metric=lambda e, p, t=None: 0, num_threads=25, display_progress=True)
evaluate = dspy.Evaluate(devset=devset, metric=top5_recall, num_threads=16, display_progress=False, max_errors=100)

eval_result = evaluate_trainset(wrap_student(react))

# append those to history
history = []
history.append({"current_react_prompt": react.react.signature.instructions, "current_extraction_prompt": react.extract.predict.signature.instructions})
history.append({"last_trajectories": get_last_traces(eval_result)})

current_state = {"current_react_prompt": react.react.signature.instructions, "current_extraction_prompt": react.extract.predict.signature.instructions}
# pass to reasoner and modify prompts
new_prompts = {}
with dspy.context(lm=teacher_lm):
    for i in range(5):
        reasoner = dspy.Predict(ReasonAboutTraces, seed=i)
        reasoner_result = reasoner(
            history=history,
            current_state=current_state,
        )
        new_prompts[i] = reasoner_result
        print(reasoner_result)

# evaluate these on ground truth
# eval_set_result = evaluate(react)
# print(eval_set_result)

import concurrent.futures
# with dspy.context(lm=student_lm):
#     original_score = evaluate(react).score

new_prompts[-1] = dspy.Prediction(improved_react_prompt=react.react.signature.instructions, improved_extraction_prompt=react.extract.predict.signature.instructions)

def evaluate_prompt_pair(num, reasoner_eval_result):
    instructions = "Find all Wikipedia titles relevant to verifying (or refuting) the claim."
    signature = dspy.Signature("claim -> titles: list[str]", instructions)
    react_evaluate = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=10)
    react_evaluate.react.signature.instructions = reasoner_eval_result.improved_react_prompt
    react_evaluate.extract.predict.signature.instructions = reasoner_eval_result.improved_extraction_prompt
    with dspy.context(lm=student_lm):
        rich.print("="*10,
        f"Running evaliaton number {num}",
        # f"Current prompt: {new_react.react.signature.instructions}",
        # f"Current extraction prompt: {new_react.extract.predict.signature.instructions}",
        "="*10)
        new_full_evaluate = evaluate(react_evaluate)
    return num, new_full_evaluate.score

results = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    # Dictionary mapping future to num for tracking and ordering
    future_to_num = {
        executor.submit(evaluate_prompt_pair, num, prompt): num 
        for num, prompt in new_prompts.items()
    }
    for future in concurrent.futures.as_completed(future_to_num):
        num, score = future.result()
        results[num] = score

print("="*80)

rich.print("Original prompt score: ", results[-1])
results.pop(-1)
rich.print("New prompt scores: ", results)
rich.print("Average score: ", sum(results.values()) / len(results))


# run on 20 new trajectories

Average Metric: 0.00 / 20 (0.0%): 100%|██████████| 20/20 [00:00<00:00, 163.75it/s]

2025/12/11 16:08:55 INFO dspy.evaluate.evaluate: Average Metric: 0 / 20 (0.0%)



Prediction(
    deviations_from_specification='- Repeatedly passed arguments to the finish tool, e.g., {"titles": [...]}, {"title": ...}, {"type": ...}, {"parent_company": ...}, despite the spec requiring finish to take {} only. This caused execution errors.\n- Occasionally used finish as if it were an output channel for titles rather than a signal to stop.\n- Sometimes relied solely on search snippets without confirming via lookup_wikipedia when claims were multi-part or potentially ambiguous.\n- Minor premature stopping in a few cases without verifying all atomic parts of a compound claim (e.g., not checking both entities when the claim compares them).',
    noticed_patterns='- Positive patterns:\n  - Good use of disambiguation in queries (adding years or qualifiers like "(1990 film)").\n  - Often decomposed claims into sub-parts and retrieved key entity pages (people, places, works).\n  - Used town/city pages for census numbers and line terminus pages for rail claims appropriately.

========== Running evaliaton number 0 ==========

========== Running evaliaton number 1 ==========

...

========== Running evaliaton number -1 ==========

2025/12/11 16:20:40 INFO dspy.evaluate.evaluate: Average Metric: 58.0 / 100 (58.0%)
2025/12/11 16:22:21 ERROR dspy.utils.parallelizer: Error for Example({'claim': 'This individual had called their debut four-track extended play Write.. (EP). Eddie Vedder was born before them.', 'titles': ['Nam Woo-hyun', 'Write.. (EP)', 'Eddie Vedder']}) (input_keys={'claim'}): litellm.RateLimitError: RateLimitError: OpenrouterException - Provider returned error. Set `provide_traceback=True` for traceback.
2025/12/11 16:22:23 ERROR dspy.utils.parallelizer: Error for Example({'claim': "The father of the founder of St Hugh's College, Oxford, was a bishop in the Anglican Church.", 'titles': ["St Hugh's College, Oxford", 'Elizabeth Wordsworth', 'Christopher Wordsworth']}) (input_keys={'claim'}): litellm.RateLimitError: RateLimitError: OpenrouterException - Provider returned error. Set `provide_traceback=True` for traceback.
2025/12/11 16:22:25 ERROR dspy.utils.parallelizer: Error for Example({'claim': 'The

Original prompt score:  58.0

New prompt scores: 
{3: 58.67, 2: 71.33, 4: 54.67, 0: 66.67, 1: 61.0}

Average score:  62.468

In [ ]:
# ==============
# Evaluate from 5x experiment
# ==============

# with dspy.context(lm=student_lm):
#     original_score = evaluate(react).score
prompts_with_original = new_prompts.copy()
prompts_with_original[-1] = {"improved_react_prompt": react.react.signature.instructions, "improved_extraction_prompt": react.extract.predict.signature.instructions}

def evaluate_prompt_pair(num, reasoner_result):
    new_react = react.deepcopy()
    new_react.react.signature.instructions = reasoner_result.improved_react_prompt
    new_react.extract.predict.signature.instructions = reasoner_result.improved_extraction_prompt
    with mlflow.start_run(run_name=f"evaluate_prompt_pair_{num}"):
        with dspy.context(lm=student_lm):
            rich.print("="*80,
            f"Running evaliaton number {num}",
            f"Current prompt: {new_react.react.signature.instructions}",
            f"Current extraction prompt: {new_react.extract.predict.signature.instructions}",
            "="*80)
            new_full_evaluate = evaluate_trainset(wrap_student(new_react), devset=current_trainset[:1])
    return new_full_evaluate

for num, prompt in prompts_with_original.items():
    rich.print("="*80)
    rich.print(f"Running evaliaton number {num}")
    rich.print(f"Current prompt: {prompt.improved_react_prompt}")
    rich.print(f"Current extraction prompt: {prompt.improved_extraction_prompt}")
    res = evaluate_prompt_pair(num, prompt)
    rich.print(res)
    rich.print("="*80)


================================================================================

Running evaliaton number 0

Current prompt: Find all Wikipedia titles relevant to verifying (or refuting) the claim.

You are an Agent. Input: a single field claim. You can see your past trajectory. Your goal is to use the tools to 
gather enough information so that the relevant Wikipedia titles can be extracted later.

Interaction protocol:
- On each turn, output next_thought, next_tool_name, and next_tool_args (JSON). After the tool runs, its 
observation is appended to the trajectory. Continue until you have all necessary information, then finish.
- Tools:
  (1) search_wikipedia: {'query': 'string'} — Returns top-5 results and titles of the top-5 to top-30 results.
  (2) lookup_wikipedia: {'title': 'string'} — Returns page text for the exact title.
  (3) finish: {} — Marks the task complete. No arguments allowed other than {}.

Critical rules:
- Never include any fields other than those specified for a tool. For finish, always pass {} exactly. Do not 
attempt to pass the final titles or any other data in finish.
- Do not guess page titles. If unsure, use search_wikipedia to find the canonical title, then use lookup_wikipedia 
to confirm facts.
- Prefer verifying key facts in the page text returned by lookup_wikipedia instead of relying solely on search 
snippets.
- If a lookup fails (“No Wikipedia page found”), refine your search query (add years, disambiguation terms, or 
alternate names) and try again.

Strategy:
- Decompose the claim into entities and relations. Plan which titles you need (e.g., person, work, award, location,
line/terminus, census town).
- Use search_wikipedia to identify the canonical titles, then lookup_wikipedia to confirm the specific facts needed
to verify or refute each part of the claim.
- Gather a minimal, sufficient set of titles that together address all sub-claims. Avoid extraneous pages.
- Before calling finish, quickly check:
  - Have you covered every sub-claim with at least one relevant title?
  - Are the titles canonical and unambiguous?

Output format each turn:
- next_thought: Your brief plan and reasoning for the next step(s).
- next_tool_name: One of search_wikipedia, lookup_wikipedia, or finish.
- next_tool_args: A JSON object containing only the allowed fields for that tool.

Current extraction prompt: Extract the minimal set of canonical Wikipedia page titles necessary and sufficient to 
verify or refute the claim.

Guidelines:
- Include titles for every critical entity and evidence page referenced in the gathered observations (e.g., person,
film/work, award, location/airport, railway line + terminus, census town).
- Prefer the exact canonical page titles as shown by Wikipedia (including disambiguation parentheses where 
applicable).
- Exclude extraneous or duplicate titles.
- If the claim has multiple parts, ensure titles cover each part.

Output:
- A simple list of Wikipedia titles relevant to verifying or refuting the claim.

================================================================================ Running evaliaton number 0 Current
prompt: Find all Wikipedia titles relevant to verifying (or refuting) the claim.

You are an Agent. Input: a single field claim. You can see your past trajectory. Your goal is to use the tools to 
gather enough information so that the relevant Wikipedia titles can be extracted later.

Interaction protocol:
- On each turn, output next_thought, next_tool_name, and next_tool_args (JSON). After the tool runs, its 
observation is appended to the trajectory. Continue until you have all necessary information, then finish.
- Tools:
  (1) search_wikipedia: {'query': 'string'} — Returns top-5 results and titles of the top-5 to top-30 results.
  (2) lookup_wikipedia: {'title': 'string'} — Returns page text for the exact title.
  (3) finish: {} — Marks the task complete. No arguments allowed other than {}.

Critical rules:
- Never include any fields other than those specified for a tool. For finish, always pass {} exactly. Do not 
attempt to pass the final titles or any other data in finish.
- Do not guess page titles. If unsure, use search_wikipedia to find the canonical title, then use lookup_wikipedia 
to confirm facts.
- Prefer verifying key facts in the page text returned by lookup_wikipedia instead of relying solely on search 
snippets.
- If a lookup fails (“No Wikipedia page found”), refine your search query (add years, disambiguation terms, or 
alternate names) and try again.

Strategy:
- Decompose the claim into entities and relations. Plan which titles you need (e.g., person, work, award, location,
line/terminus, census town).
- Use search_wikipedia to identify the canonical titles, then lookup_wikipedia to confirm the specific facts needed
to verify or refute each part of the claim.
- Gather a minimal, sufficient set of titles that together address all sub-claims. Avoid extraneous pages.
- Before calling finish, quickly check:
  - Have you covered every sub-claim with at least one relevant title?
  - Are the titles canonical and unambiguous?

Output format each turn:
- next_thought: Your brief plan and reasoning for the next step(s).
- next_tool_name: One of search_wikipedia, lookup_wikipedia, or finish.
- next_tool_args: A JSON object containing only the allowed fields for that tool. Current extraction prompt: 
Extract the minimal set of canonical Wikipedia page titles necessary and sufficient to verify or refute the claim.

Guidelines:
- Include titles for every critical entity and evidence page referenced in the gathered observations (e.g., person,
film/work, award, location/airport, railway line + terminus, census town).
- Prefer the exact canonical page titles as shown by Wikipedia (including disambiguation parentheses where 
applicable).
- Exclude extraneous or duplicate titles.
- If the claim has multiple parts, ensure titles cover each part.

Output:
- A simple list of Wikipedia titles relevant to verifying or refuting the claim. 
================================================================================

...

================================================================================

================================================================================

Running evaliaton number -1

AttributeError: 'dict' object has no attribute 'improved_react_prompt'

[Trace(trace_id=tr-071802ed4c7d19050a5350a3812e69ba), Trace(trace_id=tr-e570955847f4917d04f29a9e1465a1ee), Trace(trace_id=tr-c9a5754f1bb2f69158b0cf2fe7cfa81c), Trace(trace_id=tr-4b73e7fd4199b36680b3d64e77c19952), Trace(trace_id=tr-a0fea0f3d63f8fe3ade90f8fadcf8406), Trace(trace_id=tr-6ee68b7f893f14a2531141d1cf8e3c90), Trace(trace_id=tr-f0a0f421e1f8f5a087c0db38ccd4f87f)]

In [44]:
rich.print(new_prompts)

{
    0: Prediction(
    deviations_from_specification='- Repeatedly passed unsupported arguments to the finish tool (e.g., {"titles": 
[...]}, {"title": "...", "parent_company": "..."}), despite finish requiring {} only.\n- Occasionally attempted 
lookup_wikipedia with guessed or ambiguous titles without first disambiguating via search (e.g., "Lionheart (film)"
instead of searching and using "Lionheart (1990 film)").\n- At times relied solely on search snippets without 
confirming key facts via lookup_wikipedia, leading to minor reasoning slips (e.g., internally calling a species a 
genus).\n- In a few cases, prematurely finished without collecting all minimally necessary titles to verify all 
sub-claims.',
    noticed_patterns='- Positive: Usually begins with a targeted search query; switches to lookup for confirmation;
decomposes multi-part claims into sub-entities (e.g., person + award/film + location).\n- Error pattern: Passing 
data payloads in finish calls is the most common failure mode.\n- Query refinement: When a lookup fails, the agent 
often recovers by refining the search (good), but could more systematically handle disambiguation.\n- Evidence 
sufficiency: Sometimes includes only one of multiple necessary titles (e.g., entity page but not the relevant 
event/location/line page).\n- Snippet trust: Occasional over-reliance on search result snippets instead of 
verifying via page text for critical facts.',
    generic_strategy='- Parse the claim into concrete entities and relations. List which Wikipedia titles are 
likely needed to verify each relation.\n- First use search_wikipedia with precise queries (include years, 
disambiguators in parentheses) to find canonical page titles. Do not guess titles.\n- For each required title, use 
lookup_wikipedia to confirm the key fact(s). Prefer page text over search snippets for verification.\n- Handle 
ambiguity: if lookup fails or you’re unsure of the exact title, run another search with refined keywords or 
alternative phrasings.\n- Collect a minimal, sufficient set of titles that together verify or refute all parts of 
the claim (e.g., person + work + award; line + terminus + town).\n- Only when you have gathered all necessary 
titles, call finish with empty JSON {}. Never include any other fields or answers in the finish call.\n- Sanity 
checks before finishing:\n  - Are all sub-claims covered by at least one verified title each?\n  - Are titles 
canonical (exact Wikipedia page titles), non-duplicative, and scoped to the claim?',
    improved_react_prompt="Find all Wikipedia titles relevant to verifying (or refuting) the claim.\n\nYou are an 
Agent. Input: a single field claim. You can see your past trajectory. Your goal is to use the tools to gather 
enough information so that the relevant Wikipedia titles can be extracted later.\n\nInteraction protocol:\n- On 
each turn, output next_thought, next_tool_name, and next_tool_args (JSON). After the tool runs, its observation is 
appended to the trajectory. Continue until you have all necessary information, then finish.\n- Tools:\n  (1) 
search_wikipedia: {'query': 'string'} — Returns top-5 results and titles of the top-5 to top-30 results.\n  (2) 
lookup_wikipedia: {'title': 'string'} — Returns page text for the exact title.\n  (3) finish: {} — Marks the task 
complete. No arguments allowed other than {}.\n\nCritical rules:\n- Never include any fields other than those 
specified for a tool. For finish, always pass {} exactly. Do not attempt to pass the final titles or any other data
in finish.\n- Do not guess page titles. If unsure, use search_wikipedia to find the canonical title, then use 
lookup_wikipedia to confirm facts.\n- Prefer verifying key facts in the page text returned by lookup_wikipedia 
instead of relying solely on search snippets.\n- If a lookup fails (“No Wikipedia page found”), refine your search 
query (add years, disambiguation terms, or alternate names) and try again.\n\nStrategy:\n- Decompose the claim

In [ ]:
# ==============
# Initial iteration
# ==============

import rich
from typing import Any
from concurrent.futures import ThreadPoolExecutor
import concurrent
import json
from datetime import datetime

mlflow.dspy.autolog(disable=True)

# student_lm = dspy.LM("openrouter/qwen/qwen3-8b")
student_lm = dspy.LM("openai/gpt-5-nano")
teacher_lm = dspy.LM("openai/gpt-5")
dspy.configure(lm=student_lm)

class ReasonAboutTraces(dspy.Signature):
    """Given sets of trajectories and the prompts that were used to generate them
    reason about the traces and how you might improve the prompt from a strategy perspective without overfitting.
    
    Reason through the prompt evolution over time to try and construct the best possible prompt for the next iterations.
    """
    history: list[str] = dspy.InputField()
    current_state: dict[str, Any] = dspy.InputField()
    reasoning: str = dspy.OutputField()
    next_react_prompt: str = dspy.OutputField()
    next_extraction_prompt: str = dspy.OutputField()

# get 20 trajectories
def wrap_student(student_prog, seed=None):
    def wrapped_student(*args, **kwargs):
        with dspy.context(trace=[]):
            result = student_prog(*args, **kwargs, seed=seed)
            trace = dspy.settings.trace
            return [result, trace]
    return wrapped_student

def write_history(history, filename):
    with open(filename, "w") as f:
        jsons = []
        for h in history:
            # If h is not a dict but can be converted via **, do so
            try:
                jsons.append(json.dumps(h, indent=2))
            except TypeError:
                try:
                    jsons.append(json.dumps({**h}, indent=2))
                except Exception:
                    # fallback: string representation as error handling
                    jsons.append(repr(h))
        f.write('[\n' + ',\n'.join(jsons) + '\n]\n')

instructions = "Find all Wikipedia titles relevant to verifying (or refuting) the claim."
signature = dspy.Signature("claim -> titles: list[str]", instructions)
# current_react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=10)
evaluate_devset = dspy.Evaluate(devset=devset, metric=top5_recall, num_threads=25, display_progress=True, max_errors=50)

run_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
prog_save_dir = f"programs/{run_datetime}"
os.makedirs(prog_save_dir, exist_ok=True)

def run_iteration(seed):
    history = []
    programs_over_time = []
    current_react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=10)
    reasoner = dspy.Predict(ReasonAboutTraces, seed=seed)
    for i in range(4):
        rich.print("="*80,
        f"Running iteration {i}",
        f"Current prompt: {current_react.react.signature.instructions}",
        f"Current extraction prompt: {current_react.extract.predict.signature.instructions}",
        "="*80)
        programs_over_time.append(current_react.deepcopy())
        current_react.save(f"{prog_save_dir}/iter_{seed}_program_{i}.json")
        current_trainset = trainset[i*20:(i+1)*20]
        evaluate_trainset = dspy.Evaluate(devset=current_trainset, metric=lambda e, p, t=None: 0, num_threads=25, display_progress=False, provide_traceback=True)

        # create new trajectories
        with dspy.context(lm=student_lm):
            first_trainset_eval = evaluate_trainset(wrap_student(react, seed=seed))

        # append those to history
        history.append({f"trajectories_iteration_{i}": get_last_traces(first_trainset_eval)})
        history.append({f"react_prompt_for_iteration_{i}": react.react.signature.instructions, f"extraction_prompt_for_iteration_{i}": react.extract.predict.signature.instructions})

        current_state = {"current_react_prompt": react.react.signature.instructions, "current_extraction_prompt": react.extract.predict.signature.instructions}
        # pass to reasoner and modify prompts
        with dspy.context(lm=teacher_lm):
            reasoner_result = reasoner(
                history=history,
                current_state=current_state,
            )
        # append the reasoning and new prompts
        history.append({f"prompt_optimization_iteration_{i}": {**reasoner_result}})

        current_react.react.signature.instructions = reasoner_result.next_react_prompt
        current_react.extract.predict.signature.instructions = reasoner_result.next_extraction_prompt

    # final_evaluate = evaluate_devset(current_react)
    write_history(history, f"{prog_save_dir}/prompt_history_{seed}.json")

    return programs_over_time


# with ThreadPoolExecutor(max_workers=8) as executor:
#     executor.map(run_iteration, range(5))

results = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    # Dictionary mapping future to num for tracking and ordering
    future_to_num = {
        executor.submit(run_iteration, i): i 
        for i in range(5)
    }
    for future in concurrent.futures.as_completed(future_to_num):
        num = future_to_num[future]
        programs = future.result()
        results[num] = programs
    
# import json

final_programs = [results[i][-1] for i in range(5)]

def evaluate_program(program):
    with dspy.context(lm=student_lm):
        final_evaluate = evaluate_devset(program)
    return final_evaluate.score

instructions = "Find all Wikipedia titles relevant to verifying (or refuting) the claim."
signature = dspy.Signature("claim -> titles: list[str]", instructions)
original_react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=10)
original_score = evaluate_program(original_react).score

scores = []
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    future_to_num = {
        executor.submit(evaluate_program, program): i 
        for i, program in enumerate(final_programs)
    }
    for future in concurrent.futures.as_completed(future_to_num):
        num = future_to_num[future]
        scores.append(future.result())


rich.print("="*80)
rich.print("Original prompt score: ", original_score)
rich.print("New prompt scores: ", scores)
rich.print("Average score: ", sum(scores) / len(scores))


# run on 20 new trajectories

================================================================================ Running iteration 0 Current 
prompt: Find all Wikipedia titles relevant to verifying (or refuting) the claim.

You are an Agent. In each episode, you will be given the fields `claim` as input. And you can see your past 
trajectory so far.
Your goal is to use one or more of the supplied tools to collect any necessary information for producing `titles`.

To do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when 
finishing the task.
After each tool call, you receive a resulting observation, which gets appended to your trajectory.

When writing next_thought, you may reason about the current situation and plan for future steps.
When selecting the next_tool_name and its next_tool_args, the tool must be one of:

(1) search_wikipedia, whose description is <desc>Returns top-5 results and then the titles of the top-5 to top-30 
results.</desc>. It takes arguments {'query': {'type': 'string'}}.
(2) lookup_wikipedia, whose description is <desc>Returns the text of the Wikipedia page, if it exists.</desc>. It 
takes arguments {'title': {'type': 'string'}}.
(3) finish, whose description is <desc>Marks the task as complete. That is, signals that all information for 
producing the outputs, i.e. `titles`, are now available to be extracted.</desc>. It takes arguments {}.
When providing `next_tool_args`, the value inside the field must be in JSON format Current extraction prompt: Find 
all Wikipedia titles relevant to verifying (or refuting) the claim. 
================================================================================

================================================================================ Running iteration 0 Current 
prompt: Find all Wikipedia titles relevant to verifying (or refuting) the claim.

You are an Agent. In each episode, you will be given the fields `claim` as input. And you can see your past 
trajectory so far.
Your goal is to use one or more of the supplied tools to collect any necessary information for producing `titles`.

To do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when 
finishing the task.
After each tool call, you receive a resulting observation, which gets appended to your trajectory.

When writing next_thought, you may reason about the current situation and plan for future steps.
When selecting the next_tool_name and its next_tool_args, the tool must be one of:

(1) search_wikipedia, whose description is <desc>Returns top-5 results and then the titles of the top-5 to top-30 
results.</desc>. It takes arguments {'query': {'type': 'string'}}.
(2) lookup_wikipedia, whose description is <desc>Returns the text of the Wikipedia page, if it exists.</desc>. It 
takes arguments {'title': {'type': 'string'}}.
(3) finish, whose description is <desc>Marks the task as complete. That is, signals that all information for 
producing the outputs, i.e. `titles`, are now available to be extracted.</desc>. It takes arguments {}.
When providing `next_tool_args`, the value inside the field must be in JSON format Current extraction prompt: Find 
all Wikipedia titles relevant to verifying (or refuting) the claim. 
================================================================================

================================================================================ Running iteration 0 Current 
prompt: Find all Wikipedia titles relevant to verifying (or refuting) the claim.

You are an Agent. In each episode, you will be given the fields `claim` as input. And you can see your past 
trajectory so far.
Your goal is to use one or more of the supplied tools to collect any necessary information for producing `titles`.

To do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when 
finishing the task.
After each tool call, you receive a resulting observation, which gets appended to your trajectory.

When writing next_thought, you may reason about the current situation and plan for future steps.
When selecting the next_tool_name and its next_tool_args, the tool must be one of:

(1) search_wikipedia, whose description is <desc>Returns top-5 results and then the titles of the top-5 to top-30 
results.</desc>. It takes arguments {'query': {'type': 'string'}}.
(2) lookup_wikipedia, whose description is <desc>Returns the text of the Wikipedia page, if it exists.</desc>. It 
takes arguments {'title': {'type': 'string'}}.
(3) finish, whose description is <desc>Marks the task as complete. That is, signals that all information for 
producing the outputs, i.e. `titles`, are now available to be extracted.</desc>. It takes arguments {}.
When providing `next_tool_args`, the value inside the field must be in JSON format Current extraction prompt: Find 
all Wikipedia titles relevant to verifying (or refuting) the claim. 
================================================================================

================================================================================ Running iteration 0 Current 
prompt: Find all Wikipedia titles relevant to verifying (or refuting) the claim.

You are an Agent. In each episode, you will be given the fields `claim` as input. And you can see your past 
trajectory so far.
Your goal is to use one or more of the supplied tools to collect any necessary information for producing `titles`.

To do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when 
finishing the task.
After each tool call, you receive a resulting observation, which gets appended to your trajectory.

When writing next_thought, you may reason about the current situation and plan for future steps.
When selecting the next_tool_name and its next_tool_args, the tool must be one of:

(1) search_wikipedia, whose description is <desc>Returns top-5 results and then the titles of the top-5 to top-30 
results.</desc>. It takes arguments {'query': {'type': 'string'}}.
(2) lookup_wikipedia, whose description is <desc>Returns the text of the Wikipedia page, if it exists.</desc>. It 
takes arguments {'title': {'type': 'string'}}.
(3) finish, whose description is <desc>Marks the task as complete. That is, signals that all information for 
producing the outputs, i.e. `titles`, are now available to be extracted.</desc>. It takes arguments {}.
When providing `next_tool_args`, the value inside the field must be in JSON format Current extraction prompt: Find 
all Wikipedia titles relevant to verifying (or refuting) the claim. 
================================================================================

================================================================================ Running iteration 0 Current 
prompt: Find all Wikipedia titles relevant to verifying (or refuting) the claim.

You are an Agent. In each episode, you will be given the fields `claim` as input. And you can see your past 
trajectory so far.
Your goal is to use one or more of the supplied tools to collect any necessary information for producing `titles`.

To do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when 
finishing the task.
After each tool call, you receive a resulting observation, which gets appended to your trajectory.

When writing next_thought, you may reason about the current situation and plan for future steps.
When selecting the next_tool_name and its next_tool_args, the tool must be one of:

(1) search_wikipedia, whose description is <desc>Returns top-5 results and then the titles of the top-5 to top-30 
results.</desc>. It takes arguments {'query': {'type': 'string'}}.
(2) lookup_wikipedia, whose description is <desc>Returns the text of the Wikipedia page, if it exists.</desc>. It 
takes arguments {'title': {'type': 'string'}}.
(3) finish, whose description is <desc>Marks the task as complete. That is, signals that all information for 
producing the outputs, i.e. `titles`, are now available to be extracted.</desc>. It takes arguments {}.
When providing `next_tool_args`, the value inside the field must be in JSON format Current extraction prompt: Find 
all Wikipedia titles relevant to verifying (or refuting) the claim. 
================================================================================

...

2025/12/12 13:38:53 INFO dspy.evaluate.evaluate: Average Metric: 62.666666666666664 / 100 (62.7%)


================================================================================

Original prompt score:  58

New prompt scores: 
[70.33, 69.0, 73.0, 57.67, 62.67]

Average score:  66.534

In [64]:
# print(len(history))
with open("prompt_history.json", "w") as f:
    jsons = []
    for h in history:
        # If h is not a dict but can be converted via **, do so
        try:
            jsons.append(json.dumps(h, indent=2))
        except TypeError:
            try:
                jsons.append(json.dumps({**h}, indent=2))
            except Exception:
                # fallback: string representation as error handling
                jsons.append(repr(h))
    f.write('[\n' + ',\n'.join(jsons) + '\n]\n')

In [ ]:
# ==============
# Eval perf over time
# ==============
RUN_ID="20251212_130856"

run_dir = f"programs/{RUN_ID}"




In [ ]:
import concurrent.futures
with dspy.context(lm=student_lm):
    original_score = evaluate(react).score

def evaluate_prompt_pair(num, reasoner_result):
    new_react = react.deepcopy()
    new_react.react.signature.instructions = reasoner_result.improved_react_prompt
    new_react.extract.predict.signature.instructions = reasoner_result.improved_extraction_prompt
    with dspy.context(lm=student_lm):
        new_full_evaluate = evaluate(new_react)
    return num, new_full_evaluate.score

# Prefer ThreadPoolExecutor for notebook compatibility; ProcessPoolExecutor can misbehave in notebooks
results = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    # Dictionary mapping future to num for tracking and ordering
    future_to_num = {
        executor.submit(evaluate_prompt_pair, num, prompt): num 
        for num, prompt in new_prompts.items()
    }
    for future in concurrent.futures.as_completed(future_to_num):
        num, score = future.result()
        results[num] = score

In [ ]:
with dspy.context(lm=student_lm):
    initial_evaluation = evaluate(initial_react)
    